<a href="https://colab.research.google.com/github/El-Wally/Movie-Recommender/blob/wip/MoviesRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import time

**Importing the dataset**

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2022-07-19 06:05:39--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  26.3MB/s    in 5.5s    

2022-07-19 06:05:45 (27.8 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [ ]:
df = pd.read_csv('movies.csv')
df.head(50)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
df['title'][7001]

'Accident (1967)'

checking for null values

In [ ]:
df.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [ ]:
df1 = pd.read_csv('tags.csv')
df1.head(5)

,userId,movieId,tag,timestamp
0,19,2324,bittersweet,1428651158
1,19,2324,holocaust,1428651112
2,19,2324,World War II,1428651118
3,23,7075,hilarious,1378675786
4,23,7075,Underrated,1378675786


In [ ]:
df1.describe

<bound method NDFrame.describe of         userId  movieId                            tag   timestamp
0           19     2324                    bittersweet  1428651158
1           19     2324                      holocaust  1428651112
2           19     2324                   World War II  1428651118
3           23     7075                      hilarious  1378675786
4           23     7075                     Underrated  1378675786
...        ...      ...                            ...         ...
586989  247751    96655                         future  1366041126
586990  247751    96655                         robots  1366041128
586991  247753      356                        classic  1430438776
586992  247753      356  oscar (best supporting actor)  1430438776
586993  247753      356                   social drama  1430438776

[586994 rows x 4 columns]>

In [ ]:
df1['tag'].value_counts()

sci-fi                 6194
action                 4059
atmospheric            3839
based on a book        3799
comedy                 3745
                       ... 
Manny Coto                1
Darin Scott               1
Alain Berliner            1
Richard Friedenberg       1
great screenplays         1
Name: tag, Length: 45084, dtype: int64

In [ ]:
df1[df1.isnull().any(axis=1)]

,userId,movieId,tag,timestamp
119651,44258,33826,NaN,1221450908
208021,91097,123,NaN,1199450867
208022,91097,346,NaN,1199451946
208026,91097,1184,NaN,1199452261
208033,91097,1785,NaN,1199452006
208034,91097,2194,NaN,1199450677
208036,91097,2691,NaN,1199451002
208044,91097,4103,NaN,1199451920
208046,91097,4473,NaN,1199451040
208048,91097,4616,NaN,1199452441


In [ ]:
(df1['movieId']==1).sum()

548

In [ ]:
  dff1 = df1[df1['movieId'].values  == 1]['tag']
  dff1=dff1.tolist()
  dff1

['animation',
 'fun',
 'animation',
 'children',
 'toys',
 'itaege',
 'animation',
 'comedy',
 'Disney',
 'Pixar',
 'Disney',
 'fun',
 'good animation',
 'lovable characters',
 'action',
 'disney',
 'kids',
 'Pixar',
 'Tom Hanks',
 'adventure',
 'animated',
 'animation',
 'computer animation',
 'Disney',
 'funny',
 'pixar',
 'Tom Hanks',
 'toys',
 'Pixar',
 'buddy movie',
 'Tom Hanks',
 'witty',
 'cute',
 'funny',
 'story',
 'voice acting',
 'witty',
 'time travel',
 'animation',
 'children',
 'Disney',
 'Pixar',
 'children',
 'Disney',
 'fantasy',
 'pixar',
 'animation',
 'pixar',
 'Tim Allen',
 'Tom Hanks',
 'Pixar',
 'clever',
 'animation',
 'kids movie',
 'adventure',
 'animation',
 'children',
 'comedy',
 'unlikely friendships',
 'funny',
 'Pixar',
 'witty',
 'é˜®ä¸€é¸£',
 'children',
 'Pixar',
 '3D',
 'animated',
 'children',
 'comedy',
 'computer animation',
 'Disney',
 'family',
 'humorous',
 'Pixar',
 'time travel',
 'Tom Hanks',
 'kids',
 'light hearted',
 'whimsica',
 'Pixar

In [ ]:
new_df = pd.DataFrame(columns=df1.columns)
new_df.head(1)

,userId,movieId,tag,timestamp


In [ ]:
x = (list(set(df1['movieId'])))

In [ ]:
x.sort()
x

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 129,
 132,
 133,
 134,
 135,
 136,
 138,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 196,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 205,
 206

In [ ]:
"""depreciated, slower runtime than the later function"""


# start = time.time()
# for i in x:
#   df_1 = df1[df1['movieId'].values  == i]['tag']
#   entry= {'movieId':i,'tag':set(df_1)}
#   new_df=new_df.append(entry,ignore_index=True)

# new_df
# end = time.time()
# print(end - start)


'depreciated, slower runtime than the later function'

In [ ]:
df1.sort_values(by=['movieId'], inplace=True)
print(df1)
print(df1['tag'].iloc[0])

        userId  movieId          tag   timestamp
69201    29321        1       Disney  1273665156
585030  246887        1        pixar  1443906050
42826    15036        1        Pixar  1261495514
413751  173452        1        pixar  1445458804
413752  173452        1    Tim Allen  1445458849
...        ...      ...          ...         ...
308824  132513   151677       trippy  1453951839
308825  132513   151677         Yogi  1453951834
308819  132513   151677        drugs  1453951843
308818  132513   151677        dance  1453951844
411565  172056   151701  documentary  1454031135

[586994 rows x 4 columns]
Disney


In [ ]:
#df1['movieId'].tolist()
x=list(set(df1['movieId']))

In [ ]:
x.sort()

In [ ]:
x

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 129,
 132,
 133,
 134,
 135,
 136,
 138,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 196,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 205,
 206

In [ ]:
temp = 0
for i in x:

  dff1 = df1[df1['movieId'].values  == i]['tag']
  dff1=dff1.tolist()

  if(len(dff1)>0):

    df1.drop(df1.index[temp+1:temp+len(dff1)],inplace=True)
    df1['tag'].iloc[temp] = dff1
  
  temp+=1 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df1.head(10)

,userId,movieId,tag,timestamp
69201,29321,1,"[Disney, pixar, Pixar, pixar, Tim Allen, Tom H...",1273665156
426208,179664,2,"[Filmed in BC, family, fantasy, Robin Williams...",1148752833
195259,86161,3,"[Walter Matthau, old people that is actually f...",1161144194
117334,43484,4,"[revenge, CLV, chick flick, characters, charac...",1308558977
456190,185477,5,"[Touching, watched under duress, it thought it...",1425151186
340315,143264,6,"[realistic action, Al Pacino, bleak, philosoph...",1392885765
146467,54403,7,"[Harrison Ford, remake, romance, relationships...",1317021217
229490,99992,8,"[seen, author:Mark Twain, Peter Hewitt, adapte...",1354062797
260975,114930,9,"[Peter Hyams, CLV, action, Jean-Claude Van Dam...",1300403135
346824,143748,10,"[007 movies are bad., james bond, 007, James B...",1359142828


In [ ]:
df.rename(columns = {'movieId':'movId'},inplace = True)
df = pd.merge(df,df1,left_on='movId',right_on='movieId')

In [ ]:
df.head()

,movId,title,genres,userId,movieId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,29321,1,"[Disney, pixar, Pixar, pixar, Tim Allen, Tom H...",1273665156
1,2,Jumanji (1995),Adventure|Children|Fantasy,179664,2,"[Filmed in BC, family, fantasy, Robin Williams...",1148752833
2,3,Grumpier Old Men (1995),Comedy|Romance,86161,3,"[Walter Matthau, old people that is actually f...",1161144194
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,43484,4,"[revenge, CLV, chick flick, characters, charac...",1308558977
4,5,Father of the Bride Part II (1995),Comedy,185477,5,"[Touching, watched under duress, it thought it...",1425151186


In [ ]:
df = df.drop(['movieId','timestamp','userId'], axis=1)

In [ ]:
df3 = pd.read_csv('ratings.csv')
df3 = df3.drop(['timestamp','userId'], axis=1)
df3.head(10)

,movieId,rating
0,169,2.5
1,2471,3.0
2,48516,5.0
3,2571,3.5
4,109487,4.0
5,112552,5.0
6,112556,4.0
7,356,4.0
8,2394,4.0
9,2431,5.0


In [ ]:
temp = 0
wavg_rating=[]
for i in x:

  ratings = df3[df3['movieId'].values  == i]['rating']
  ratings=ratings.tolist()

  if(len(ratings)>0):

    wavg_rating.append(sum(ratings)/len(ratings))
  else:
    wavg_rating.append('nan')
  
  temp+=1 

In [ ]:
len(wavg_rating)

In [ ]:
df['avg_rating'] = wavg_rating

In [ ]:
df3.head(5)

In [ ]:
df1[df1['movieId'].values  == 1]['rating']

In [ ]:
mydf = df1.reindex(df1.columns.tolist() + ['0.5','1.0','1.5','2.0','2.5','3.0','3.5','4.0','4.5','5.0'], axis=1)

In [ ]:
df1['movieId']

In [ ]:
for i in df1['movieId']:
  print(i)

In [ ]:
mydf

In [ ]:
np.average(df1[df1['movieId'].values  == 1]['rating'])

In [ ]:
x = df1[df1['movieId'].values  == 1]['rating']

In [ ]:
c = Counter(x)
c

In [ ]:
labels = []
sizes = []
fig,ax = plt.subplots(figsize =(10, 7))
for x, y in c.items():
    labels.append(x)
    sizes.append(y)

#Plot
plt.stem(sizes[::-1])
ax.set_title("Movie Rating pie chart")

plt.show()

In [ ]:
sizes

In [ ]:
c.keys()[0]/sum(c.values())


In [ ]:

plt.hist(x)
plt.show() 


In [ ]:
plt.pie(x)
plt.show() 